In [2]:
! python.exe -m pip install --upgrade pip
! pip install pandas
! pip install numpy
! pip install openpyxl
import pandas as pd, numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print("\n*****Required libraries imported*****")


*****Required libraries imported*****


In [4]:
data = pd.read_csv(r"C:\Python\read\cases_report_All HESPL HES_ALL_(All States)_2024-01-12 (1).csv")
data.head()

,Case Id,Createdon,Scheme/Doc,Scheme/Doc GUID,Status,Docket Submitted Date,Benefit received Date,Case Organization,Case District,Citizen GUID,Citizen Name,Gender,Age,Family GUID,Family Name,HD ID,HD Name,Opsco name
0,EP05HANB,09-Sep-2023,Integrated Physical PAN Card (Central),DC0009SJ,Open,NaN,NaN,UP - All HESPL B2C,BARABANKI,PN05P6FL,Km Soni,F,25.0,PG05A5A5,Soni,Durgesh.Kumar2,Durgesh Kumar,Gyanendra Kumar
1,EP05HANK,09-Sep-2023,Integrated Physical PAN Card (Central),DC0009SJ,Open,NaN,NaN,UP - All HESPL B2C,BARABANKI,PN05P6FR,Km Soni,F,25.0,PG05A5AB,Soni,Durgesh.Kumar2,Durgesh Kumar,Gyanendra Kumar
2,EP060GLH,18-Nov-2023,Integrated Physical PAN Card (Central),DC0009SJ,Open,NaN,NaN,UP - All HESPL B2C,BARABANKI,PN067D6T,Ramkumar Ramkumar,M,20.0,PG05M98K,Ramkumar,Durgesh.Kumar2,Durgesh Kumar,Gyanendra Kumar
3,EP05LCIG,03-Oct-2023,Integrated Physical PAN Card (Central),DC0009SJ,Open,NaN,NaN,UP - All HESPL B2C,BARABANKI,PN05SL1C,Gauri Shankar,M,18.0,PG05DIO7,Shankar,Sarvesh.Kumar2,Sarvesh Kumar,Gyanendra Kumar
4,EP05NHGR,16-Oct-2023,Integrated Physical PAN Card (Central),DC0009SJ,Open,NaN,NaN,UP - All HESPL B2C,BARABANKI,PN060KTI,Rajesh Kumar,M,30.0,PG05FIKD,Kumar,Sarvesh.Kumar2,Sarvesh Kumar,Gyanendra Kumar


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2180 entries, 0 to 2179
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Case Id                2180 non-null   object 
 1   Createdon              2179 non-null   object 
 2   Scheme/Doc             2179 non-null   object 
 3   Scheme/Doc GUID        2179 non-null   object 
 4   Status                 2179 non-null   object 
 5   Docket Submitted Date  1265 non-null   object 
 6   Benefit received Date  1258 non-null   object 
 7   Case Organization      2179 non-null   object 
 8   Case District          2179 non-null   object 
 9   Citizen GUID           2179 non-null   object 
 10  Citizen Name           2179 non-null   object 
 11  Gender                 2179 non-null   object 
 12  Age                    2179 non-null   float64
 13  Family GUID            2179 non-null   object 
 14  Family Name            2179 non-null   object 
 15  HD I

In [6]:
data.drop(index=list(data[(data["Status"]=="Case Aborted") | (data["Status"]=="Application rejected")].index),inplace=True)
data.drop(index=data[data['Status'].isna()].index,inplace=True)
data['Status'].replace(to_replace="Data complete", value="Open",inplace=True)
data.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2150 entries, 0 to 2149
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  2150 non-null   int64  
 1   Case Id                2150 non-null   object 
 2   Createdon              2150 non-null   object 
 3   Scheme/Doc             2150 non-null   object 
 4   Scheme/Doc GUID        2150 non-null   object 
 5   Status                 2150 non-null   object 
 6   Docket Submitted Date  1265 non-null   object 
 7   Benefit received Date  1258 non-null   object 
 8   Case Organization      2150 non-null   object 
 9   Case District          2150 non-null   object 
 10  Citizen GUID           2150 non-null   object 
 11  Citizen Name           2150 non-null   object 
 12  Gender                 2150 non-null   object 
 13  Age                    2150 non-null   float64
 14  Family GUID            2150 non-null   object 
 15  Fami

In [7]:
data['Status'].value_counts()

Status
Scheme/Document received    1258
Open                         885
Docket submitted               7
Name: count, dtype: int64

In [8]:
only_YC = data[((data['Scheme/Doc']=="Yojana_Card") | (data['Scheme/Doc']=="Yojana_Card - Lost/Replace")) & (data['Status']=="Scheme/Document received")]
only_YC = pd.pivot_table(only_YC, index=['Citizen GUID', 'Citizen Name'], values='Case Id', aggfunc='count').reset_index()
only_YC.head()

,Citizen GUID,Citizen Name,Case Id
0,PN01VPUV,Manpreet Kaur,1
1,PN01VXSM,Rekha Rekha,1
2,PN01WRMK,Pradeep Kumar,1
3,PN028DCN,Gyanendra Kumar,1
4,PN02CK8X,Kamlesh Kamlesh,1


In [27]:
only_BR = pd.pivot_table(data[data['Status']=='Scheme/Document received'], index='Citizen GUID', columns='Scheme/Doc', values='Case Id', aggfunc='count', fill_value=0).reset_index()
only_BR = only_BR[only_BR['Yojana_Card'] == 1].reset_index(drop=True)
only_BR['Grand Total'] = only_BR.drop(columns=['Citizen GUID']).sum(axis=1)
only_BR.tail()

Scheme/Doc,Citizen GUID,Ayushman Bharat - Sarbat Sehat Bima Yojana_PB,Digital Health ID card (Central),Domicile Certificate,E-Shram Card (Central),FSSAI Registration _Registration for Food MSME,Integrated Physical PAN Card (Central),MP-Aadhar seeding to Samagra,MP-Caste Certificate,MP-Disability Certificate,MP-Domicile Certificate,MP-Driving Licence,MP-Income Certificate,MP-Sambal Card,Member Addition to Ration Card_UP,PM Kisan E-KYC_UP,Pradhan Mantri Jan Arogya Yojana (Ayushman Bharat)_MP,Pradhan Mantri Jan Arogya Yojana (Ayushman Bharat)_UP,Registration with National Career Services (Central),UP-BOCW Construction Labour Card,UP-Caste Certificate,UP-Income Certificate,Udyam Registration for MSME,Updation/Correction in Voter ID (Central),Vaccine Registration_UP,Yojana_Card,Grand Total
967,PN03PI60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
968,PN03PI99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
969,PN03PK5P,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
970,PN03PKKX,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
971,PN03PKZS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [28]:
only_BR.head()

Scheme/Doc,Citizen GUID,Ayushman Bharat - Sarbat Sehat Bima Yojana_PB,Digital Health ID card (Central),Domicile Certificate,E-Shram Card (Central),FSSAI Registration _Registration for Food MSME,Integrated Physical PAN Card (Central),MP-Aadhar seeding to Samagra,MP-Caste Certificate,MP-Disability Certificate,MP-Domicile Certificate,MP-Driving Licence,MP-Income Certificate,MP-Sambal Card,Member Addition to Ration Card_UP,PM Kisan E-KYC_UP,Pradhan Mantri Jan Arogya Yojana (Ayushman Bharat)_MP,Pradhan Mantri Jan Arogya Yojana (Ayushman Bharat)_UP,Registration with National Career Services (Central),UP-BOCW Construction Labour Card,UP-Caste Certificate,UP-Income Certificate,Udyam Registration for MSME,Updation/Correction in Voter ID (Central),Vaccine Registration_UP,Yojana_Card,Grand Total
0,PN01VPUV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,PN01VXSM,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2,PN01WRMK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
3,PN028DCN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2
4,PN02CK8X,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [33]:
only_BR[only_BR['Yojana_Card'] == 1]['Yojana_Card'].count()
only_BR[only_BR['Yojana_Card'] == 1]['Yojana_Card'].sum()

34

In [30]:
only_BR[only_BR['Grand Total'] == 3]

Scheme/Doc,Citizen GUID,Ayushman Bharat - Sarbat Sehat Bima Yojana_PB,Digital Health ID card (Central),Domicile Certificate,E-Shram Card (Central),FSSAI Registration _Registration for Food MSME,Integrated Physical PAN Card (Central),MP-Aadhar seeding to Samagra,MP-Caste Certificate,MP-Disability Certificate,MP-Domicile Certificate,MP-Driving Licence,MP-Income Certificate,MP-Sambal Card,Member Addition to Ration Card_UP,PM Kisan E-KYC_UP,Pradhan Mantri Jan Arogya Yojana (Ayushman Bharat)_MP,Pradhan Mantri Jan Arogya Yojana (Ayushman Bharat)_UP,Registration with National Career Services (Central),UP-BOCW Construction Labour Card,UP-Caste Certificate,UP-Income Certificate,Udyam Registration for MSME,Updation/Correction in Voter ID (Central),Vaccine Registration_UP,Yojana_Card,Grand Total
103,PN02WD6R,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,3


In [37]:
Citizen_Sch_ratio = pd.DataFrame({'Citizen':['with only YC','with YC + 1 Scheme','with YC + 3 Scheme','with YC + 3 Scheme'],
                                  'Total Citizens':[only_BR[only_BR['Yojana_Card'] == 1]['Yojana_Card'].count(),
                                                    only_BR[only_BR['Grand Total'] == 2]['Grand Total'].count(),
                                                    only_BR[only_BR['Grand Total'] == 3]['Grand Total'].count(),
                                                    only_BR[only_BR['Grand Total'] == 4]['Grand Total'].count()],
                                  'Total Applications':[only_BR[only_BR['Yojana_Card'] == 1]['Yojana_Card'].sum(),
                                                        only_BR[only_BR['Grand Total'] == 2]['Grand Total'].sum(),
                                                        only_BR[only_BR['Grand Total'] == 3]['Grand Total'].sum(),
                                                        only_BR[only_BR['Grand Total'] == 4]['Grand Total'].sum()]})

Citizen_Sch_ratio

,Citizen,Total Citizens,Total Applications
0,with only YC,972,972
1,with YC + 1 Scheme,17,34
2,with YC + 3 Scheme,1,3
3,with YC + 3 Scheme,0,0
